In [2]:
import pandas as pd
from tabulate import tabulate

# Load the data
file_path = '/Users/vrishfish/Mens-Soccer-Stats-1/Datasets/player-combined-data.csv'  
data = pd.read_csv(file_path)

# Calculate accuracy percentages
data['Pass Accuracy (%)'] = (data['total_passes_completed'] / data['total_passes']) * 100
data['Long Pass Accuracy (%)'] = (data['long_passes_completed'] / data['long_passes']) * 100
data['Cross Accuracy (%)'] = (data['accurate_crosses'] / data['crosses']) * 100
data['Dribble Accuracy (%)'] = (data['successful_dribbles'] / data['dribbles']) * 100
data['Foul Percentage (%)'] = (data['fouls_drawn'] / data['offensive_duels']) * 100
# data['Offensive Duels Won (%)'] = (data['offensive_duels_won'] / data['offensive_duels']) * 100

# Handle division by zero or NaN
accuracy_features = [
    'Pass Accuracy (%)', 'Long Pass Accuracy (%)', 'Cross Accuracy (%)',
    'Dribble Accuracy (%)', 'Foul Percentage (%)'
]
data[accuracy_features] = data[accuracy_features].fillna(0)

# Group by player and position to aggregate statistics
# Include only numeric columns for grouping
numeric_columns = data.select_dtypes(include='number').columns.tolist()
player_position_stats = data.groupby(['player_name', 'Position'])[numeric_columns].mean().reset_index()

# Calculate quartiles for accuracy metrics
quartiles = player_position_stats[accuracy_features].quantile([0.25, 0.75]).T
quartiles.columns = ['25th_percentile', '75th_percentile']

# Identify strengths and weaknesses
def evaluate_strengths_weaknesses(row):
    strengths = []
    weaknesses = []
    
    for feature in accuracy_features:
        if row[feature] >= quartiles.loc[feature, '75th_percentile']:
            strengths.append(feature)
        elif row[feature] <= quartiles.loc[feature, '25th_percentile']:
            weaknesses.append(feature)
    
    return pd.Series([', '.join(strengths), ', '.join(weaknesses)], index=['Strengths', 'Weaknesses'])

# Apply evaluation to determine strengths and weaknesses
player_position_stats[['Strengths', 'Weaknesses']] = player_position_stats.apply(evaluate_strengths_weaknesses, axis=1)

# Filter out rows where Position is 0
player_position_stats = player_position_stats[player_position_stats['Position'] != '0']


# Save results to a CSV
output_file = 'player_strengths_weaknesses.csv'
player_position_stats[['player_name', 'Position', 'Strengths', 'Weaknesses']].to_csv(output_file, index=False)

# Display a preview of the results
print(tabulate(player_position_stats[['player_name', 'Position', 'Strengths', 'Weaknesses']], headers='keys', tablefmt='grid', showindex=False))



+-------------------+-----------------------------+----------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+
| player_name       | Position                    | Strengths                                                                                                | Weaknesses                                                                                               |
+===================+=============================+==========================================================================================================+==========================================================================================================+
| A. Adalsteinsson  | CB                          | Long Pass Accuracy (%)                                                                                   | Cross Accuracy (%), Dribble Accuracy (%), F

*Comparing stats of winning vs losing teams*

taking the teams which won and lost and comparing the statistics between them

In [4]:


import pandas as pd
import matplotlib.pyplot as plt

# Drop goalkeeper stats and create a cleaned DataFrame
df = data.drop(columns=[
    'gk_stat_conceded_goals', 'gk_stat_xcg', 'gk_stat_shots_against', 
    'gk_stat_saves', 'gk_stat_reflex_saves', 'gk_stat_box_exits', 
    'gk_stat_passes_to_gk', 'gk_stat_passes_to_gk_completed', 
    'gk_stat_goal_kicks_attempted', 'gk_stat_short_goal_kicks', 
    'gk_stat_long_goal_kicks', 'year'
])

matches = df['Match']  # Replace 'Match' with the actual column name containing match data

# Function to determine the winner
def determine_winner(match):
    try:
        # Split the match string into parts
        parts = match.rsplit(' ', 1)  # Split into teams and scores
        teams = parts[0].split(' - ')  # Split teams
        scores = list(map(int, parts[1].split(':')))  # Split scores and convert to integers
        
        if scores[0] > scores[1]:
            return f"Winner: {teams[0]}"
        elif scores[0] < scores[1]:
            return f"Winner: {teams[1]}"
        else:
            return "Draw"
    except (IndexError, ValueError):
        # Handle cases with missing or malformed data
        return "Invalid Match Format"

# Apply the function to the match column
df['Result'] = matches.apply(determine_winner)

# Display the updated DataFrame
print(df[['Match', 'Result']])




# Filter matches won and lost
matches_won = df[df['Result'].str.contains('Winner')]  # Matches the team won
matches_lost = df[~df['Result'].str.contains('Winner') & ~df['Result'].str.contains('Draw')]  # Matches the team lost

# Group player stats by 'team' for matches won and lost
# Selecting only numerical columns for grouping
numerical_columns = df.select_dtypes(include=['number']).columns
player_stats_won = matches_won.groupby('team')[numerical_columns].mean()  # Mean stats for matches won
player_stats_lost = matches_lost.groupby('team')[numerical_columns].mean()  # Mean stats for matches lost

# Combine stats for comparison
comparison = player_stats_won.subtract(player_stats_lost, fill_value=0)  # Difference in stats
comparison['Match Count (Won)'] = matches_won['team'].value_counts()  # Count of matches won
comparison['Match Count (Lost)'] = matches_lost['team'].value_counts()  # Count of matches lost

# Display comparison DataFrame
print("Comparison of stats between matches won and lost:")
print(comparison)

# Save results to a CSV
output_file = 'team_winning_losing_stats.csv'
comparison.to_csv(output_file, index=False)

                                                  Match  \
0     Maryland College Park Terrapins - UCLA Bruins 0:6   
1                  UCLA Bruins - Washington Huskies 0:1   
2                    UCLA Bruins - Indiana Hoosiers 1:2   
3     Maryland College Park Terrapins - UCLA Bruins 1:1   
4             Rutgers Scarlet Knights - UCLA Bruins 0:3   
...                                                 ...   
8630      Penn State Nittany Lion - Cornell Big Red 2:3   
8631  Penn State Nittany Lion - St. Bonaventure Bonn...   
8632  Penn State Nittany Lion - Pittsburgh Panthers 1:2   
8633  Penn State Nittany Lion - Army West Point Blac...   
8634       Clemson Tigers - Penn State Nittany Lion 3:0   

                               Result  
0                 Winner: UCLA Bruins  
1          Winner: Washington Huskies  
2            Winner: Indiana Hoosiers  
3                                Draw  
4                 Winner: UCLA Bruins  
...                               ...  
8630       